# Практическое задание 4
## КМБО-01-20, Гребнев Никита, вариант 5 

Исходные данные: 
- Набор данных - [Video game sales](https://www.kaggle.com/gregorut/videogamesales)
- Тип классификатора - LogisticRegression (логистическая регрессия)
- Классификация по столбцу - Platform (PS2 – класс 0, остальные уровни – класс 1)

### Установливаем необходимые библиотеки:

In [ ]:
!pip install pandas
!pip install sklearn

### Считываем данныеиз файла, преобразуем поле Platform: `0`, если там было значение `PS2` , иначе `1`

In [2]:
import pandas
import numpy as np
data = pandas.read_csv('vgsales.csv', index_col='Rank')

# исключаем столбец Name как поле с уникальными значениями
data_sel = data.loc[:, data.columns.isin(['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])]
data_sel = data_sel.dropna()
data_sel['Platform'] = np.where(data_sel['Platform'] == 'PS2', 0, 1)

### Разделяем данные: отдельно `Platform`, отдельно все остальные данные

In [3]:
Platform = data_sel.loc[:, data_sel.columns.isin(['Platform'])]
X = data_sel.loc[:, data_sel.columns.isin(['Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])]

### Преобразуем текстовые данные в числовые 

In [4]:
my_set = set(X.Genre)
i = 0
for item in my_set:
    X['Genre'] = X['Genre'].replace(item, i)
    i = i + 1
    
i = 0
my_set2 = set(X.Publisher)
for item in my_set2:
    X['Publisher'] = X['Publisher'].replace(item, i)
    i = i + 1

<ipython-input-4-86c2fc005784>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Genre'] = X['Genre'].replace(item, i)
<ipython-input-4-86c2fc005784>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Publisher'] = X['Publisher'].replace(item, i)


In [5]:
X

,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Rank,,,,,,,
1,2006.0,5,64,41.49,29.02,3.77,8.46
2,1985.0,2,64,29.08,3.58,6.81,0.77
3,2008.0,0,64,15.85,12.88,3.79,3.31
4,2009.0,5,64,15.75,11.01,3.28,2.96
5,1996.0,7,64,11.27,8.89,10.22,1.00
...,...,...,...,...,...,...,...
16596,2002.0,2,491,0.01,0.00,0.00,0.00
16597,2003.0,6,471,0.01,0.00,0.00,0.00
16598,2008.0,0,213,0.00,0.00,0.00,0.00


### Разделяем данные на обучающую и тестовую выборку

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X, Platform, test_size=.33, random_state=5)

### Выключаем предупреждения, потому что дальше их будет слишком много, и я не знаю, как их устранить 

In [7]:
import warnings
warnings.filterwarnings('ignore')

### Построим классификатор логистической регрессии

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)

#logreg_cv = LogisticRegression(random_state=5).fit(x_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']})

### Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [9]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='recall'))))

f1:0.9268441803573799
precision:0.8646316630799724
recall:0.9987075925169815


### Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации:

In [10]:
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [50, 100, 150],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}
RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': ['auto'],
                         'n_estimators': [50, 100, 150]})

### Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [12]:
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='recall'))))

f1:0.9659419892832387
precision:0.938509249885534
recall:0.9952627964811998


### Сравним точности классификаторов
| Значение сравнения | LogisticRegression | Random Forest
|:--- | :---: | :---:
|f1 | 0.92 | 0.96
| precision | 0.86 | 0.93
| recall | 0.99 | 0.99

Видим, что показатели у Random Forest лучше